In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import cuml 
import cupy as cp
import cudf as cd
import seaborn as sns

ModuleNotFoundError: No module named 'cuml'

In [5]:
df = cd.read_csv("../input/house-price/train.csv")

In [6]:
df.head(10)

In [7]:
df.info()

In [8]:
df.shape

In [9]:
null_column = []
for col in df.columns:
    if df[col].isnull().sum()>=1:
        null_column.append(col)

In [10]:
df_train = df.drop(null_column,axis = 1)

In [11]:
df_train.head(4)

In [12]:
df_train.info()

In [13]:
final_df = df_train[['OverallQual', 'GrLivArea', 'YearBuilt', 'YearRemodAdd', 'TotalBsmtSF', '1stFlrSF', 'FullBath']]
final_df.head()

In [14]:
X = final_df
Y = df_train['SalePrice']

In [15]:
X

In [16]:
from cuml.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X_scaled.as_matrix(),Y,test_size = 0.25,random_state = 42)

In [18]:
algorithm = ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']

In [19]:
from cuml import LinearRegression
err = 9999999999
final_lr = None
for alg in algorithm:
    print("Algorithm:")
    print(alg)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = alg)
    reg = lr.fit(X_train,Y_train)
    preds = lr.predict(X_test)
    Y_test=Y_test.astype(cp.float64)
    print("MSE:")
    print(cuml.metrics.regression.mean_squared_error(Y_test,preds))
    if err>=cuml.metrics.regression.mean_squared_error(Y_test,preds):
        err = cuml.metrics.regression.mean_squared_error(Y_test,preds)
        final_lr = lr
    print("R2 Score:")
    print(cuml.metrics.regression.r2_score(Y_test,preds))
    print("MAE:")
    print(cuml.metrics.regression.mean_absolute_error(Y_test,preds))

In [20]:
test = cd.read_csv('../input/house-price/test.csv')
test.head()
test_id = test[['Id']]

In [21]:
test = test[['OverallQual', 'GrLivArea', 'YearBuilt', 'YearRemodAdd', 'TotalBsmtSF', '1stFlrSF', 'FullBath']]
test=test.fillna(0)
test.head()

In [22]:
pred = final_lr.predict(test)

In [23]:
type(test_id)
test_id = test_id.to_pandas()

In [24]:
type(pred)

In [25]:
import pandas as pd
output = pd.DataFrame()


In [26]:
output["Id"] = test_id

In [27]:
output["SalePrice"] = pd.DataFrame(pred)

In [29]:
output.to_csv('submission.csv', index=False)